<a href="https://colab.research.google.com/github/Meziane228Kechabia/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,687 kB]

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-01 18:45:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.17MB/s    in 0.2s    

2020-11-01 18:45:58 (5.17 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Read in data from url
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
grocery_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
grocery_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [7]:
# Create new dataFrame when filter applys to total_votes greater or equal to 20
filtered_votes_df = grocery_df.filter(grocery_df['total_votes'] >= 20)
filtered_votes_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2622293|R1OAZUG90XPU2U|B00JQ47DO2|     268145085|Epic Bison Bacon ...|         Grocery|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...| 2015-08-31|
|         US|   12244195|R23VPTUJ8I8NR5|B00XWQ2F2K|     138780411|Anna and Sarah Or...|         Grocery|          1|    

In [9]:
# Create dataframe where helpful_votes/total_votes>= 50%
helpful_over_ttl_df = filtered_votes_df.filter(filtered_votes_df['helpful_votes'] / filtered_votes_df['total_votes'] >= 0.5)
helpful_over_ttl_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2622293|R1OAZUG90XPU2U|B00JQ47DO2|     268145085|Epic Bison Bacon ...|         Grocery|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...| 2015-08-31|
|         US|   12244195|R23VPTUJ8I8NR5|B00XWQ2F2K|     138780411|Anna and Sarah Or...|         Grocery|          1|    

In [10]:
# Create dataFrame where review was written paid
paid_df = helpful_over_ttl_df.filter(helpful_over_ttl_df["vine"] == "Y")
paid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   34773146|R3MKO875WGIEBJ|B00N9IMNKA|      61108636|Vita Coco Organic...|         Grocery|          5|           34|         37|   Y|                N|My Top Coconut Oi...|I'm very strict w...| 2015-08-05|
|         US|   35093059|R3OPNTK61FQ7MP|B00S0AOIFA|     713485594|    V8 Veggie Blends|         Grocery|          5|    

In [11]:
# Create dataFrame where review was written unpaid
unpaid_df = helpful_over_ttl_df.filter(helpful_over_ttl_df['vine'] == 'N')
unpaid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2622293|R1OAZUG90XPU2U|B00JQ47DO2|     268145085|Epic Bison Bacon ...|         Grocery|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...| 2015-08-31|
|         US|   12244195|R23VPTUJ8I8NR5|B00XWQ2F2K|     138780411|Anna and Sarah Or...|         Grocery|          1|    

In [28]:
# Determine the number of reviews.
paid_review = paid_df.count()
unpaid_review = unpaid_df.count()

# Determine the total review.
ttl_review = paid_review + unpaid_review

# Determine the number of five star.
ttl_five_star_paid = paid_df[paid_df['star_rating'] == 5].count()
ttl_five_star_unpaid = unpaid_df[unpaid_df['star_rating'] == 5].count()

# Determine the % of five stars.
prcntg_five_star_paid = ttl_five_star_paid / paid_review
prcntg_five_star_unpaid = ttl_five_star_unpaid / unpaid_review

In [13]:
paid_df.count()

61

In [14]:
unpaid_df.count()

28287

In [29]:
#So spDF.select('colname').show()
grocery_df.select('vine').count()
#grocery_df.select('vine'=="N").count()



2402458

In [39]:
# Determine the number of vine reviews
from pyspark.sql import functions as f
grocery_df.select(f.sum(f.when(f.col('vine') == 'Y', 1)).alias('count')).show(truncate=False)
#grocery_df.select(f.sum(f.when(f.col('vine') == 'N', 1)).alias('count')).show(truncate=False)

+-----+
|count|
+-----+
|16612|
+-----+



In [40]:
# Determine the number of non vine reviews
grocery_df.select(f.sum(f.when(f.col('vine') == 'N', 1)).alias('count')).show(truncate=False)

+-------+
|count  |
+-------+
|2385819|
+-------+



In [36]:
# How many vine reviews were 5 stars
ttl_five_star_paid

20

In [38]:
# How many non nine review were 5 stars
ttl_five_star_unpaid

15689

In [41]:
# % of 5 stars vine review 
prcntg_five_star_paid



0.32786885245901637

In [42]:
# % of 5 stars non-vine review 
prcntg_five_star_unpaid

0.5546364054159154